In [1]:
#Import Libraries
import nltk

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [3]:
#Import training data in train
train = pd.read_csv(
    filepath_or_buffer='train.dat', 
    header=0, 
    sep='\t')

In [4]:
train.head()

,sentiment,Text
0,2,she had another song out recently but it didn'...
1,2,wellity! i think ima gonna clean my room :| Ga...
2,2,Bug in damn cod4 system link. My map pack isnt...
3,8,"weeooow, i feel fat.... i remember when i used..."
4,7,mmmm now you're speaking my language! (Unfortu...


In [5]:
train.describe()

,sentiment
count,31000.000000
mean,4.348097
std,3.114712
min,1.000000
25%,2.000000
50%,3.000000
75%,7.000000
max,11.000000


In [6]:
train.groupby('sentiment').describe()

Text                                                               
          count unique                                                top freq
sentiment                                                                     
1          4257   4249                                         I miss you    3
2          7006   6979        I am lost. Please help me find a good home.    9
3          7131   7074                                             Thanks    8
4          3141   3095  I just received a mothers day card from my lov...   10
5            96     96  If you don't want to come then don't come. JEE...    1
6           617    616  Hi I have uploaded 5 completely new Chinese Le...    2
7          1759   1758  Thought you might be interested in @TweetPhoto...    2
8          1458   1458  Morning Twitterland! Countdown to TCI begins a...    1
9          1072   1072  i hate i hate i hate i hate i hate i hate moth...    1
10         4311   4280                                       good morning    4
11          149    149                  I am sooooooo bored in textiles !    1

In [7]:
#Convert the text in training data to string
train['Text']=train['Text'].astype(str)

In [8]:
train['length'] = train['Text'].apply(len)

In [9]:
train.head()

,sentiment,Text,length
0,2,she had another song out recently but it didn'...,66
1,2,wellity! i think ima gonna clean my room :| Ga...,126
2,2,Bug in damn cod4 system link. My map pack isnt...,54
3,8,"weeooow, i feel fat.... i remember when i used...",120
4,7,mmmm now you're speaking my language! (Unfortu...,129


In [10]:
import string


In [11]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [12]:
from nltk.corpus import stopwords


In [13]:
# Clean the data using Porter Stemmer to normalize the data for similar words with different spellings to mean the same
from nltk.stem import PorterStemmer
porter_stemmer = PorterStemmer()
def stem_sentences(sentence):
    tokens = sentence.split(" ")
    stemmed_tokens = [porter_stemmer.stem(token) for token in tokens]
    return ' '.join(stemmed_tokens)

train['Text']=train['Text'].apply(stem_sentences)

In [14]:
train['Text'].head(10)

0    she had anoth song out recent but it didn't ge...
1    wellity! i think ima gonna clean my room :| ga...
2    bug in damn cod4 system link. My map pack isnt...
3    weeooow, i feel fat.... i rememb when i use to...
4    mmmm now you'r speak my language! (unfortunate...
5                              really?? I feel special
6            don't you procrastinate!   (like I do...)
7    ive been want 2 but will be gone 4 2 mo. &amp;...
8     sushi two night in a row isn't that wrong is it?
9          haha i get my senior licens next friday!!!!
Name: Text, dtype: object

In [15]:
#Remove numerical data from the test file
import re
for i in range(len(train)):
    train['Text'][i] = re.sub('\d', '', train['Text'][i])


C:\Users\kashi\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [16]:
train['Text'].head(10)

0    she had anoth song out recent but it didn't ge...
1    wellity! i think ima gonna clean my room :| ga...
2    bug in damn cod system link. My map pack isnt ...
3    weeooow, i feel fat.... i rememb when i use to...
4    mmmm now you'r speak my language! (unfortunate...
5                              really?? I feel special
6            don't you procrastinate!   (like I do...)
7    ive been want  but will be gone   mo. &amp; ha...
8     sushi two night in a row isn't that wrong is it?
9          haha i get my senior licens next friday!!!!
Name: Text, dtype: object

In [17]:
#Preprocess the data - remove punctuation, remove stopwords and return clean words in lower case
def text_process(mess):
    """
    1. Remove Punctuation
    2. remove stopwords
    3. return list of clean words
    """
    nopunc = [char for char in mess if char not in string.punctuation]
    nopunc = ''.join(nopunc)
    return [word for word in nopunc.split() if word.lower() not in stopwords.words('english') ]



In [18]:
train['Text'].head()

0    she had anoth song out recent but it didn't ge...
1    wellity! i think ima gonna clean my room :| ga...
2    bug in damn cod system link. My map pack isnt ...
3    weeooow, i feel fat.... i rememb when i use to...
4    mmmm now you'r speak my language! (unfortunate...
Name: Text, dtype: object

In [19]:
from sklearn.feature_extraction.text import CountVectorizer

In [20]:
from sklearn.feature_extraction.text import TfidfTransformer

In [21]:
from sklearn.model_selection import train_test_split

In [42]:
#Split the data into training and testing data
msg_train, msg_text, label_train, label_text =train_test_split(train['Text'],train['sentiment'], test_size=0.15)

In [23]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

In [24]:
from sklearn.metrics import classification_report

In [25]:
from sklearn.svm import SVC

In [25]:
#use Pipeline method to build the predictive model
#Classification using SVC, c=100, gamma - 0.001
#Fit the model and predict the outcome for test data
pipeline =Pipeline([
    ('bow',CountVectorizer(analyzer=text_process)),
    ('tfidf', TfidfTransformer()),
    ('Classifier',SVC(C =100,kernel='linear', gamma = 0.001))
])
pipeline.fit(msg_train,label_train)
predictions1 = pipeline.predict(msg_text)


In [26]:
print(classification_report(label_text, predictions1))

             precision    recall  f1-score   support

          1       0.26      0.31      0.28       453
          2       0.32      0.39      0.35       677
          3       0.34      0.39      0.36       719
          4       0.34      0.30      0.32       326
          5       0.00      0.00      0.00        10
          6       0.17      0.09      0.12        53
          7       0.06      0.06      0.06       165
          8       0.09      0.06      0.07       130
          9       0.28      0.13      0.18       115
         10       0.31      0.22      0.26       428
         11       0.40      0.08      0.14        24

avg / total       0.29      0.29      0.29      3100



In [27]:
#use Pipeline method to build the predictive model
#Classification using SVC, c=100, gamma - 0.01
#Fit the model and predict the outcome for test data
pipeline1 =Pipeline([
    ('bow',CountVectorizer(analyzer=text_process)),
    ('tfidf', TfidfTransformer()),
    ('Classifier',SVC(C =100,kernel='linear', gamma = 0.01))
])
pipeline1.fit(msg_train,label_train)
predictions11 = pipeline1.predict(msg_text)
print(classification_report(label_text, predictions11))

             precision    recall  f1-score   support

          1       0.26      0.31      0.28       453
          2       0.32      0.39      0.35       677
          3       0.34      0.39      0.36       719
          4       0.34      0.30      0.32       326
          5       0.00      0.00      0.00        10
          6       0.17      0.09      0.12        53
          7       0.06      0.06      0.06       165
          8       0.09      0.06      0.07       130
          9       0.28      0.13      0.18       115
         10       0.31      0.22      0.26       428
         11       0.40      0.08      0.14        24

avg / total       0.29      0.29      0.29      3100



In [28]:
#use Pipeline method to build the predictive model
#Classification using SVC, c=100, gamma - 0.1
#Fit the model and predict the outcome for test data
pipeline2 =Pipeline([
    ('bow',CountVectorizer(analyzer=text_process)),
    ('tfidf', TfidfTransformer()),
    ('Classifier',SVC(C =100,kernel='linear', gamma = 0.1))
])
pipeline2.fit(msg_train,label_train)
predictions12 = pipeline2.predict(msg_text)
print(classification_report(label_text, predictions12))

             precision    recall  f1-score   support

          1       0.26      0.31      0.28       453
          2       0.32      0.39      0.35       677
          3       0.34      0.39      0.36       719
          4       0.34      0.30      0.32       326
          5       0.00      0.00      0.00        10
          6       0.17      0.09      0.12        53
          7       0.06      0.06      0.06       165
          8       0.09      0.06      0.07       130
          9       0.28      0.13      0.18       115
         10       0.31      0.22      0.26       428
         11       0.40      0.08      0.14        24

avg / total       0.29      0.29      0.29      3100



In [29]:
#use Pipeline method to build the predictive model
#Classification using SVC, c=10, gamma - 0.1
#Fit the model and predict the outcome for test data
pipeline3 =Pipeline([
    ('bow',CountVectorizer(analyzer=text_process)),
    ('tfidf', TfidfTransformer()),
    ('Classifier',SVC(C =10,kernel='linear', gamma = 0.1))
])
pipeline3.fit(msg_train,label_train)
predictions13 = pipeline3.predict(msg_text)
print(classification_report(label_text, predictions13))

             precision    recall  f1-score   support

          1       0.26      0.29      0.28       453
          2       0.33      0.41      0.36       677
          3       0.33      0.38      0.35       719
          4       0.37      0.35      0.36       326
          5       0.00      0.00      0.00        10
          6       0.15      0.08      0.10        53
          7       0.09      0.07      0.08       165
          8       0.08      0.03      0.04       130
          9       0.28      0.13      0.18       115
         10       0.29      0.24      0.26       428
         11       0.40      0.08      0.14        24

avg / total       0.29      0.30      0.29      3100



In [25]:
#use Pipeline method to build the predictive model
#Classification using SVC, c=1, gamma - 0.01
#Fit the model and predict the outcome for test data
pipeline4 =Pipeline([
    ('bow',CountVectorizer(analyzer=text_process)),
    ('tfidf', TfidfTransformer()),
    ('Classifier',SVC(C =1,kernel='linear', gamma = 0.01))
])
pipeline4.fit(msg_train,label_train)
predictions14 = pipeline4.predict(msg_text)
print(classification_report(label_text, predictions14))

             precision    recall  f1-score   support

          1       0.31      0.21      0.25       400
          2       0.36      0.52      0.42       699
          3       0.35      0.55      0.43       702
          4       0.48      0.35      0.40       313
          5       0.00      0.00      0.00        11
          6       0.00      0.00      0.00        71
          7       0.47      0.03      0.06       210
          8       0.00      0.00      0.00       142
          9       0.50      0.19      0.27       108
         10       0.34      0.34      0.34       427
         11       0.00      0.00      0.00        17

avg / total       0.34      0.36      0.33      3100



C:\Users\kashi\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [26]:
#use Pipeline method to build the predictive model
#Classification using SVC, c=1, gamma - 0.1
#Fit the model and predict the outcome for test data
pipeline5 =Pipeline([
    ('bow',CountVectorizer(analyzer=text_process)),
    ('tfidf', TfidfTransformer()),
    ('Classifier',SVC(C =1,kernel='linear', gamma = 0.1))
])
pipeline5.fit(msg_train,label_train)
predictions15 = pipeline5.predict(msg_text)
print(classification_report(label_text, predictions15))

             precision    recall  f1-score   support

          1       0.31      0.21      0.25       400
          2       0.36      0.52      0.42       699
          3       0.35      0.55      0.43       702
          4       0.48      0.35      0.40       313
          5       0.00      0.00      0.00        11
          6       0.00      0.00      0.00        71
          7       0.47      0.03      0.06       210
          8       0.00      0.00      0.00       142
          9       0.50      0.19      0.27       108
         10       0.34      0.34      0.34       427
         11       0.00      0.00      0.00        17

avg / total       0.34      0.36      0.33      3100



C:\Users\kashi\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [29]:
msg_train, msg_text, label_train, label_text =train_test_split(train['Text'],train['sentiment'], test_size=0.1)

In [30]:
#use Pipeline method to build the predictive model
#Classification using SVC, c=5, gamma - 0.1
#Fit the model and predict the outcome for test data
pipeline6 =Pipeline([
    ('bow',CountVectorizer(analyzer=text_process)),
    ('tfidf', TfidfTransformer()),
    ('Classifier',SVC(C =5,kernel='linear', gamma = .1))
])
pipeline6.fit(msg_train,label_train)
predictions16 = pipeline6.predict(msg_text)
print(classification_report(label_text, predictions16))

             precision    recall  f1-score   support

          1       0.30      0.33      0.31       447
          2       0.36      0.41      0.39       728
          3       0.34      0.40      0.37       707
          4       0.44      0.37      0.40       306
          5       0.00      0.00      0.00         7
          6       0.00      0.00      0.00        66
          7       0.17      0.09      0.12       170
          8       0.09      0.04      0.06       142
          9       0.37      0.23      0.29       103
         10       0.34      0.37      0.36       412
         11       0.00      0.00      0.00        12

avg / total       0.32      0.34      0.33      3100



In [31]:
#use Pipeline method to build the predictive model
#Classification using SVC, c=10, gamma - 0.1
#Fit the model and predict the outcome for test data
pipeline7 =Pipeline([
    ('bow',CountVectorizer(analyzer=text_process)),
    ('tfidf', TfidfTransformer()),
    ('Classifier',SVC(C =10,kernel='linear', gamma = .1))
])
pipeline7.fit(msg_train,label_train)
predictions17 = pipeline7.predict(msg_text)
print(classification_report(label_text, predictions17))

             precision    recall  f1-score   support

          1       0.29      0.34      0.31       447
          2       0.36      0.39      0.38       728
          3       0.34      0.40      0.37       707
          4       0.43      0.36      0.39       306
          5       0.00      0.00      0.00         7
          6       0.00      0.00      0.00        66
          7       0.14      0.10      0.12       170
          8       0.12      0.06      0.08       142
          9       0.35      0.22      0.27       103
         10       0.33      0.33      0.33       412
         11       0.00      0.00      0.00        12

avg / total       0.32      0.33      0.32      3100



In [43]:
msg_train, msg_text, label_train, label_text =train_test_split(train['Text'],train['sentiment'], test_size=0.15)
#use Pipeline method to build the predictive model
#Classification using SVC, c=1, gamma - 1
#Fit the model and predict the outcome for test data
pipeline8 =Pipeline([
    ('bow',CountVectorizer(analyzer=text_process)),
    ('tfidf', TfidfTransformer()),
    ('Classifier',SVC(C =1,kernel='linear', gamma = 1))
])
pipeline8.fit(msg_train,label_train)
predictions18 = pipeline8.predict(msg_text)
print(classification_report(label_text, predictions18))

             precision    recall  f1-score   support

          1       0.35      0.26      0.30       658
          2       0.35      0.47      0.40      1029
          3       0.36      0.53      0.43      1086
          4       0.53      0.39      0.45       491
          5       0.00      0.00      0.00        10
          6       0.00      0.00      0.00        95
          7       0.35      0.03      0.06       267
          8       0.29      0.02      0.04       205
          9       0.45      0.17      0.25       148
         10       0.33      0.35      0.34       634
         11       0.00      0.00      0.00        27

avg / total       0.36      0.36      0.34      4650



C:\Users\kashi\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [48]:
msg_train, msg_text, label_train, label_text =train_test_split(train['Text'],train['sentiment'], test_size=0.12)
#use Pipeline method to build the predictive model
#Classification using SVC, c=1, gamma - 1
#Fit the model and predict the outcome for test data
pipeline23 =Pipeline([
    ('bow',CountVectorizer(analyzer=text_process)),
    ('tfidf', TfidfTransformer()),
    ('Classifier',SVC(C =1,kernel='linear', gamma = 1))
])
pipeline23.fit(msg_train,label_train)
predictions33 = pipeline23.predict(msg_text)
print(classification_report(label_text, predictions33))

             precision    recall  f1-score   support

          1       0.33      0.21      0.25       550
          2       0.35      0.52      0.42       833
          3       0.37      0.54      0.44       892
          4       0.52      0.39      0.44       353
          5       0.00      0.00      0.00        12
          6       0.00      0.00      0.00        63
          7       0.44      0.03      0.06       206
          8       0.30      0.02      0.03       168
          9       0.43      0.17      0.25       132
         10       0.35      0.36      0.36       488
         11       0.00      0.00      0.00        23

avg / total       0.36      0.37      0.34      3720



C:\Users\kashi\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [53]:
msg_train, msg_text, label_train, label_text =train_test_split(train['Text'],train['sentiment'], test_size=0.15)
#use Pipeline method to build the predictive model
#Classification using SVC, c=1, gamma - 1
#Fit the model and predict the outcome for test data
pipeline24 =Pipeline([
    ('bow',CountVectorizer(analyzer=text_process)),
    ('tfidf', TfidfTransformer()),
    ('Classifier',SVC(C =1,kernel='linear', gamma = 0.1))
])
pipeline24.fit(msg_train,label_train)
predictions34 = pipeline24.predict(msg_text)
print(classification_report(label_text, predictions34))

             precision    recall  f1-score   support

          1       0.40      0.27      0.32       663
          2       0.35      0.50      0.41      1035
          3       0.37      0.55      0.44      1096
          4       0.48      0.39      0.43       445
          5       0.00      0.00      0.00        12
          6       0.00      0.00      0.00        92
          7       0.47      0.03      0.06       256
          8       0.22      0.01      0.02       224
          9       0.51      0.19      0.28       150
         10       0.39      0.39      0.39       657
         11       0.00      0.00      0.00        20

avg / total       0.38      0.38      0.35      4650



C:\Users\kashi\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [59]:
msg_train, msg_text, label_train, label_text =train_test_split(train['Text'],train['sentiment'], test_size=0.12)
#use Pipeline method to build the predictive model
#Classification using SVC, c=1, gamma - 1
#Fit the model and predict the outcome for test data
pipeline25 =Pipeline([
    ('bow',CountVectorizer(analyzer=text_process)),
    ('tfidf', TfidfTransformer()),
    ('Classifier',SVC(C =1,kernel='linear', gamma = 1))
])
pipeline25.fit(msg_train,label_train)
predictions35 = pipeline25.predict(msg_text)
print(classification_report(label_text, predictions35))

             precision    recall  f1-score   support

          1       0.38      0.26      0.31       515
          2       0.35      0.49      0.41       843
          3       0.36      0.54      0.43       848
          4       0.56      0.44      0.49       404
          5       0.00      0.00      0.00        10
          6       0.00      0.00      0.00        73
          7       0.32      0.04      0.08       211
          8       0.11      0.01      0.01       145
          9       0.50      0.19      0.28       130
         10       0.37      0.36      0.37       525
         11       0.00      0.00      0.00        16

avg / total       0.37      0.38      0.35      3720



C:\Users\kashi\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [24]:
#use Pipeline method to build the predictive model
#Classification using SVC, c=1, gamma - 0.01
#Fit the model and predict the outcome for test data
pipeline9 =Pipeline([
    ('bow',CountVectorizer(analyzer=text_process)),
    ('tfidf', TfidfTransformer()),
    ('Classifier',SVC(C =1,kernel='linear', gamma = 1))
])
pipeline9.fit(msg_train,label_train)
predictions19 = pipeline9.predict(msg_text)
print(classification_report(label_text, predictions19))

             precision    recall  f1-score   support

          1       0.37      0.24      0.29       425
          2       0.34      0.49      0.40       651
          3       0.33      0.53      0.41       696
          4       0.55      0.39      0.45       365
          5       0.00      0.00      0.00         8
          6       0.00      0.00      0.00        60
          7       0.50      0.06      0.11       179
          8       0.10      0.01      0.01       129
          9       0.33      0.12      0.17        94
         10       0.40      0.37      0.38       478
         11       0.00      0.00      0.00        15

avg / total       0.37      0.37      0.34      3100



C:\Users\kashi\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [27]:
#use Pipeline method to build the predictive model
#Classification using SVC, c=1, gamma - 10
#Fit the model and predict the outcome for test data
pipeline10 =Pipeline([
    ('bow',CountVectorizer(analyzer=text_process)),
    ('tfidf', TfidfTransformer()),
    ('Classifier',SVC(C =1,kernel='linear', gamma = 10))
])
pipeline10.fit(msg_train,label_train)
predictions20 = pipeline10.predict(msg_text)
print(classification_report(label_text, predictions20))

             precision    recall  f1-score   support

          1       0.34      0.24      0.28       873
          2       0.34      0.47      0.39      1378
          3       0.35      0.55      0.43      1394
          4       0.50      0.37      0.43       656
          5       0.00      0.00      0.00        19
          6       0.00      0.00      0.00       126
          7       0.44      0.04      0.07       374
          8       0.20      0.02      0.03       284
          9       0.51      0.19      0.28       212
         10       0.37      0.37      0.37       863
         11       0.00      0.00      0.00        21

avg / total       0.36      0.36      0.33      6200



C:\Users\kashi\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [27]:
#use Pipeline method to build the predictive model
#Classification using SVC, c=1, gamma - 100
#Fit the model and predict the outcome for test data
pipeline11 =Pipeline([
    ('bow',CountVectorizer(analyzer=text_process)),
    ('tfidf', TfidfTransformer()),
    ('Classifier',SVC(C =1,kernel='linear', gamma = 100))
])
pipeline11.fit(msg_train,label_train)
predictions21 = pipeline11.predict(msg_text)
print(classification_report(label_text, predictions21))

             precision    recall  f1-score   support

          1       0.37      0.24      0.29       425
          2       0.34      0.49      0.40       651
          3       0.33      0.53      0.41       696
          4       0.55      0.39      0.45       365
          5       0.00      0.00      0.00         8
          6       0.00      0.00      0.00        60
          7       0.50      0.06      0.11       179
          8       0.10      0.01      0.01       129
          9       0.33      0.12      0.17        94
         10       0.40      0.37      0.38       478
         11       0.00      0.00      0.00        15

avg / total       0.37      0.37      0.34      3100



C:\Users\kashi\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [28]:
#use Pipeline method to build the predictive model
#Classification using SVC, c=1, gamma - 1000
#Fit the model and predict the outcome for test data
pipeline12 =Pipeline([
    ('bow',CountVectorizer(analyzer=text_process)),
    ('tfidf', TfidfTransformer()),
    ('Classifier',SVC(C =1,kernel='linear', gamma = 1000))
])
pipeline12.fit(msg_train,label_train)
predictions22 = pipeline12.predict(msg_text)
print(classification_report(label_text, predictions22))

             precision    recall  f1-score   support

          1       0.37      0.24      0.29       425
          2       0.34      0.49      0.40       651
          3       0.33      0.53      0.41       696
          4       0.55      0.39      0.45       365
          5       0.00      0.00      0.00         8
          6       0.00      0.00      0.00        60
          7       0.50      0.06      0.11       179
          8       0.10      0.01      0.01       129
          9       0.33      0.12      0.17        94
         10       0.40      0.37      0.38       478
         11       0.00      0.00      0.00        15

avg / total       0.37      0.37      0.34      3100



C:\Users\kashi\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [27]:
#use Pipeline method to build the predictive model
#Classification using SVC, c=1, gamma - 100
#Fit the model and predict the outcome for test data
pipeline13 =Pipeline([
    ('bow',CountVectorizer(analyzer=text_process)),
    ('tfidf', TfidfTransformer()),
    ('Classifier',SVC(C =1,kernel='linear', gamma = 100))
])
pipeline13.fit(msg_train,label_train)
predictions23 = pipeline13.predict(msg_text)
print(classification_report(label_text, predictions23))

             precision    recall  f1-score   support

          1       0.32      0.22      0.26       429
          2       0.33      0.48      0.39       675
          3       0.36      0.55      0.43       707
          4       0.55      0.39      0.46       331
          5       0.00      0.00      0.00         6
          6       0.00      0.00      0.00        68
          7       0.30      0.03      0.06       192
          8       0.25      0.01      0.03       146
          9       0.49      0.20      0.28        92
         10       0.34      0.35      0.34       437
         11       0.00      0.00      0.00        17

avg / total       0.35      0.36      0.33      3100



C:\Users\kashi\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [29]:
#use Pipeline method to build the predictive model
#Classification using SVC, c=1, gamma - 200
#Fit the model and predict the outcome for test data
pipeline14 =Pipeline([
    ('bow',CountVectorizer(analyzer=text_process)),
    ('tfidf', TfidfTransformer()),
    ('Classifier',SVC(C =1,kernel='linear', gamma = 200))
])
pipeline14.fit(msg_train,label_train)
predictions24 = pipeline14.predict(msg_text)
print(classification_report(label_text, predictions24))

             precision    recall  f1-score   support

          1       0.32      0.22      0.26       429
          2       0.33      0.48      0.39       675
          3       0.36      0.55      0.43       707
          4       0.55      0.39      0.46       331
          5       0.00      0.00      0.00         6
          6       0.00      0.00      0.00        68
          7       0.30      0.03      0.06       192
          8       0.25      0.01      0.03       146
          9       0.49      0.20      0.28        92
         10       0.34      0.35      0.34       437
         11       0.00      0.00      0.00        17

avg / total       0.35      0.36      0.33      3100



C:\Users\kashi\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [30]:
#use Pipeline method to build the predictive model
#Classification using SVC, c=1, gamma - 2000
#Fit the model and predict the outcome for test data
pipeline15 =Pipeline([
    ('bow',CountVectorizer(analyzer=text_process)),
    ('tfidf', TfidfTransformer()),
    ('Classifier',SVC(C =1,kernel='linear', gamma = 2000))
])
pipeline15.fit(msg_train,label_train)
predictions25 = pipeline15.predict(msg_text)
print(classification_report(label_text, predictions25))

             precision    recall  f1-score   support

          1       0.32      0.22      0.26       429
          2       0.33      0.48      0.39       675
          3       0.36      0.55      0.43       707
          4       0.55      0.39      0.46       331
          5       0.00      0.00      0.00         6
          6       0.00      0.00      0.00        68
          7       0.30      0.03      0.06       192
          8       0.25      0.01      0.03       146
          9       0.49      0.20      0.28        92
         10       0.34      0.35      0.34       437
         11       0.00      0.00      0.00        17

avg / total       0.35      0.36      0.33      3100



C:\Users\kashi\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [31]:
#use Pipeline method to build the predictive model
#Classification using SVC, c=10, gamma - 200
#Fit the model and predict the outcome for test data
pipeline16 =Pipeline([
    ('bow',CountVectorizer(analyzer=text_process)),
    ('tfidf', TfidfTransformer()),
    ('Classifier',SVC(C =10,kernel='linear', gamma = 200))
])
pipeline16.fit(msg_train,label_train)
predictions26 = pipeline16.predict(msg_text)
print(classification_report(label_text, predictions26))

             precision    recall  f1-score   support

          1       0.26      0.29      0.27       429
          2       0.31      0.39      0.35       675
          3       0.35      0.41      0.38       707
          4       0.43      0.36      0.39       331
          5       0.00      0.00      0.00         6
          6       0.00      0.00      0.00        68
          7       0.10      0.06      0.08       192
          8       0.11      0.07      0.08       146
          9       0.24      0.17      0.20        92
         10       0.34      0.30      0.32       437
         11       0.00      0.00      0.00        17

avg / total       0.30      0.31      0.30      3100



In [103]:
# filter number 
#use Pipeline method to build the predictive model
#Classification using SVC, c=1, gamma - 1
#Fit the model and predict the outcome for test data

pipeline17 =Pipeline([
    ('bow',CountVectorizer(analyzer=text_process)),
    ('tfidf', TfidfTransformer()),
    ('Classifier',SVC(C =1,kernel='linear', gamma = 1))
])
pipeline17.fit(msg_train,label_train)
predictions27 = pipeline17.predict(msg_text)
print(classification_report(label_text, predictions27))

             precision    recall  f1-score   support

          1       0.36      0.25      0.30       428
          2       0.35      0.50      0.41       689
          3       0.37      0.55      0.44       733
          4       0.48      0.40      0.44       313
          5       0.00      0.00      0.00        11
          6       0.00      0.00      0.00        71
          7       0.24      0.03      0.05       185
          8       0.25      0.01      0.01       141
          9       0.52      0.20      0.29       110
         10       0.38      0.37      0.37       407
         11       0.00      0.00      0.00        12

avg / total       0.36      0.37      0.34      3100



C:\Users\kashi\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [50]:
msg_train, msg_text, label_train, label_text =train_test_split(train['Text'],train['sentiment'], test_size=0.15)

In [113]:
#use Pipeline method to build the predictive model
#Classification using SVC, c=1, gamma - 1
#Fit the model and predict the outcome for test data
pipeline18 =Pipeline([
    ('bow',CountVectorizer(analyzer=text_process)),
    ('tfidf', TfidfTransformer()),
    ('Classifier',SVC(C =1,kernel='linear', gamma = 1))
])
pipeline18.fit(msg_train,label_train)
predictions28 = pipeline18.predict(msg_text)
print(classification_report(label_text, predictions28))

             precision    recall  f1-score   support

          1       0.34      0.23      0.27       846
          2       0.36      0.51      0.42      1405
          3       0.35      0.53      0.42      1415
          4       0.53      0.39      0.45       623
          5       0.00      0.00      0.00        18
          6       0.00      0.00      0.00       132
          7       0.29      0.04      0.07       335
          8       0.09      0.01      0.01       280
          9       0.48      0.17      0.25       222
         10       0.35      0.34      0.35       894
         11       0.00      0.00      0.00        30

avg / total       0.35      0.37      0.34      6200



C:\Users\kashi\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [145]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
msg_train, msg_text, label_train, label_text =train_test_split(train['Text'],train['sentiment'], test_size=0.1)

In [185]:
#use Pipeline method to build the predictive model
#Classification using Ranfom Forest Classification, n_estimators=100
#Fit the model and predict the outcome for test data
pipeline19 =Pipeline([
    ('bow',CountVectorizer(analyzer=text_process)),
    ('tfidf', TfidfTransformer()),
    ('Classifier',RandomForestClassifier(n_estimators=100))
])
pipeline19.fit(msg_train,label_train)
predictions29 = pipeline19.predict(msg_text)
print(classification_report(label_text, predictions29))

             precision    recall  f1-score   support

          1       0.32      0.20      0.25       411
          2       0.35      0.49      0.40       698
          3       0.36      0.56      0.44       711
          4       0.52      0.40      0.45       351
          5       0.00      0.00      0.00         8
          6       0.00      0.00      0.00        55
          7       0.22      0.03      0.05       170
          8       0.22      0.03      0.05       148
          9       0.39      0.19      0.26       110
         10       0.37      0.32      0.34       428
         11       0.00      0.00      0.00        10

avg / total       0.35      0.36      0.34      3100



C:\Users\kashi\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [29]:
#use Pipeline method to build the predictive model
#Classification using Ranfom Forest Classification, n_estimators=3000
#Fit the model and predict the outcome for test data
pipeline20 =Pipeline([
    ('bow',CountVectorizer(analyzer=text_process)),
    ('tfidf', TfidfTransformer()),
    ('Classifier',RandomForestClassifier(n_estimators=3000))
])
pipeline20.fit(msg_train,label_train)
predictions30 = pipeline20.predict(msg_text)
print(classification_report(label_text, predictions30))

             precision    recall  f1-score   support

          1       0.35      0.19      0.24       873
          2       0.34      0.47      0.39      1378
          3       0.35      0.59      0.44      1394
          4       0.48      0.39      0.43       656
          5       0.00      0.00      0.00        19
          6       0.00      0.00      0.00       126
          7       0.44      0.05      0.09       374
          8       0.09      0.01      0.01       284
          9       0.45      0.19      0.27       212
         10       0.34      0.32      0.33       863
         11       0.00      0.00      0.00        21

avg / total       0.35      0.36      0.33      6200



C:\Users\kashi\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [28]:
clf = RandomForestClassifier(n_estimators=1000)
params = {'class_weight':['balanced']}

In [ ]:
#use Pipeline method to build the predictive model
#Classification using Ranfom Forest Classification, n_estimators=3000
#Fit the model and predict the outcome for test dat
from sklearn.model_selection import GridSearchCV
pipeline21 =Pipeline([
    ('bow',CountVectorizer(analyzer=text_process)),
    ('tfidf', TfidfTransformer()),
    ('Classifier',GridSearchCV(estimator=clf, param_grid=params, cv=5))
])
pipeline21.fit(msg_train,label_train)
predictions31 = pipeline31.predict(msg_text)
print(classification_report(label_text, predictions31))

In [40]:
from sklearn.ensemble import GradientBoostingClassifier
#use Pipeline method to build the predictive model
#Classification using Gradient Booster
#Fit the model and predict the outcome for test dat

pipeline21 =Pipeline([
    ('bow',CountVectorizer(analyzer=text_process)),
    ('tfidf', TfidfTransformer()),
    ('Classifier',GradientBoostingClassifier(learning_rate=.1,n_estimators=200))
])
pipeline21.fit(msg_train,label_train)


NameError: name 'pipeline31' is not defined

In [41]:
predictions31 = pipeline21.predict(msg_text)
print(classification_report(label_text, predictions31))

             precision    recall  f1-score   support

          1       0.40      0.21      0.27       873
          2       0.36      0.37      0.36      1378
          3       0.32      0.63      0.42      1394
          4       0.49      0.38      0.42       656
          5       0.00      0.00      0.00        19
          6       0.01      0.01      0.01       126
          7       0.26      0.04      0.07       374
          8       0.11      0.02      0.04       284
          9       0.30      0.22      0.25       212
         10       0.36      0.27      0.31       863
         11       0.00      0.00      0.00        21

avg / total       0.34      0.34      0.32      6200



In [30]:
#Read test data into test
testfile = pd.read_csv(
    filepath_or_buffer='test.dat', 
    header=None, 
    sep='\t',skip_blank_lines=False)

In [31]:
#convert test data into string
testfile[0]=testfile[0].astype(str)

In [32]:
testfile.head(5)

,0
0,Your still thinking? Cheer up Buddy )
1,"forecast in sf for friday, may 29: 66 degrees."
2,driving past Tattered Cover w/o stopping prom...
3,Wowzer! It's very windy. Not good for my aller...
4,Start of diet today I think I have to face I ...


In [33]:
#Clean the data using Porter Stemmer
from nltk.stem import PorterStemmer
porter_stemmer = PorterStemmer()
def stem_sentences(sentence):
    tokens = sentence.split(" ")
    stemmed_tokens = [porter_stemmer.stem(token) for token in tokens]
    return ' '.join(stemmed_tokens)

testfile[0]=testfile[0].apply(stem_sentences)

In [34]:
#Remove numbers from the data
import re
for i in range(len(testfile)):
    testfile[0][i] = re.sub('\d', '', testfile[0][i])

In [54]:
testfile.head()

,0
0,your still thinking? cheer up buddi )
1,"forecast in sf for friday, may : degrees."
2,drive past tatter cover w/o stop promis mysel...
3,wowzer! it' veri windy. not good for my allerg...
4,start of diet today I think I have to face I ...


In [60]:
#Predict the sentiment for test data
predictions = pipeline25.predict(testfile[0])

In [61]:
predictions[0:30]

array([10,  2,  3, 10,  2,  1,  7,  3,  1,  1,  3, 10,  3,  2,  3,  9,  3,
        4,  4,  1,  1,  2,  3,  3,  2,  4,  3,  2,  2,  3], dtype=int64)

In [62]:
#Save the test data 
np.savetxt('pred25.dat',predictions,fmt = '%d')